In [7]:
from http.server import HTTPServer, SimpleHTTPRequestHandler, BaseHTTPRequestHandler, HTTPStatus
import http.client, json, threading

class MyHandler(BaseHTTPRequestHandler):
    def do_PUT(self):
        self.close_connection = True
        msg = self.rfile.readline().decode('utf-8')
        event_handler.fire(self.server.event, msg.strip())
        self.send_response(200)
        self.end_headers()
        
    def log_message(self, *args):
        pass
    
class MyServer(HTTPServer):
    def __init__(self, port=12389, event="PUT_MSG"):
        super().__init__(("localhost", port), MyHandler)
        self.event = event
        
class ServerThread(threading.Thread):
    def __init__(self, port=12389, event="PUT_MSG"):
        super().__init__(None)
        self.port = port
        self.event = event
        self.server = None
    
    def run(self):
        self.server = MyServer(self.port, self.event)
        self.server.serve_forever()
        
    def shutdown(self):
        if self.server:
            self.server.shutdown()
            self.join(10)
            print("Thread Shutdown")
        self.server = None
        
class EventHandler():
    def __init__(self):
        self.events = {}
        
    def fire(self, event, *args):
        handlers = self.events.get(event, ())
        for h in handlers:
            h(event, *args)
        
    def register(self, event, handler):
        if not callable(handler):
            raise TypeError(type(handler))
        if event not in self.events:
            self.events[event] = []
        handlers = self.events[event]
        if handler in handlers:
            return
        handlers.append(handler)
event_handler = EventHandler()

In [8]:
import tkinter as tk
import tkinter.ttk as ttk

In [9]:
def update_label(idx):
    def update(_, msg):
        labels[idx].config(text=msg)
    return update
    

root = tk.Tk()
mf = ttk.Frame(root)
mf.grid()
labels = []
servers = []
ports = 12346, 24321, 12389
for i in range(3):
    l = ttk.Label(mf, text="No Msg")
    l.config(width=200)
    pl = ttk.Label(mf, text="Port %d: "%ports[i])
    pl.grid(row=i, column=0)
    l.grid(row=i, column=1)
    labels.append(l)
    p = ports[i]
    event = "MSG_%d"%p
    s = ServerThread(p, event)
    event_handler.register(event, update_label(i))
    s.start()
    servers.append(s)
root.mainloop()
event_handler.events.clear()
for s in servers:
    s.shutdown()

Thread Shutdown
Thread Shutdown
Thread Shutdown


In [ ]:
BaseHTTPRequestHandler??